# Welcome to the live demo!

This notebook is an easy way to upload your own hand drawn spiral. Follow the instructions in the cells below to upload your own photos and run them through the best performing network!

In [23]:
# Run this cell to load proper imports and constants
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from PIL import Image

# constants
model_dir = '/content/'

In [ ]:
# Run this cell to mount this notebook into your GDrive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Run this cell to download the model from GitHub to be run locally
!wget https://raw.githubusercontent.com/minneker/transfer-learning-project/main/inceptionv3_fc_optim

In [35]:
# Run this cell to load utility functions
def load_model():
    inception_fc = models.inception_v3(pretrained=True, aux_logits=False)
    num_ftrs = inception_fc.fc.in_features
    inception_fc.fc = nn.Linear(num_ftrs, 2)
    inception_fc.load_state_dict(torch.load(model_dir + 'inceptionv3_fc_optim'))
    return inception_fc

model = load_model()

def model_inference(img_path):
    input_image = Image.open(img_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
      output = model(input_batch)

    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
    probs = torch.nn.functional.softmax(output[0], dim=0)

    if probs[0] > probs[1]:
        print('Prediction: ', 'Control (healthy spiral)')
    else:
        print('Prediction: ', 'PwP (Parkinson\'s spiral)')

In [ ]:
# Run this cell to run inference on your image of choice
# You may specify a photo from your GDrive using it's path. 
# I suggest drawing a spiral on paper and then uploading to  
# your GDrive, then just add the path below and run!

my_img = '/path/to/your/drive/img'

model_inference(my_img)